# Neural Network
Het doel van deze opdracht is om een basisbegrip van neurale netwerken te ontwikkelen door een eenvoudig neuraal netwerk te implementeren, zonder gebruik te maken van backpropagation en gradient descent. Er dient voor het trainen van het model een simpel algoritme gebruikt te worden. Je moet de code zelf goed kan uitleggen.

In [1]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Input

from torch import nn
import torch
from torch.utils.data import DataLoader, TensorDataset

## Database connectie

In [2]:
# database name
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse'}

In [3]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')


# Create a cursor from the connection
export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
else:
    print("Connection with database is not established")
    raise Exception("Connection with database is not established")

Connection with database is established


## Tabellen inlezen
We hebben de volgende tabellen nodig:
- Order_details
- Product
- Order

In [4]:
product_query = "SELECT * FROM Product"

product_result = export_cursor.execute(product_query)
product_fetch = product_result.fetchall()
product_columns = [column[0] for column in product_result.description]
product = pd.DataFrame.from_records(product_fetch, columns=product_columns)

# Dropping current_value and last_updated columns
product = product.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

product

,PRODUCT_SK,PRODUCT_number,PRODUCT_name_product,PRODUCT_description_description,PRODUCT_image_image,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en
0,1,1,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",P01CE1CG1.jpg,2011-02-15,4.0000,0.3300,EN,4.3300,1,Camping Equipment,1,Cooking Gear
1,2,10,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",P10CE1CG1.jpg,2011-02-15,10.0000,0.4000,EN,10.4000,1,Camping Equipment,1,Cooking Gear
2,3,100,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,P100OP4FA17.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,17,First Aid
3,4,101,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,P101GE5IR18.jpg,2019-12-15,305.5400,0.4300,EN,305.9700,5,Golf Equipment,18,Irons
4,5,102,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,P102GE5IR18.jpg,2019-10-12,380.9500,0.5100,EN,381.4600,5,Golf Equipment,18,Irons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,95,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv...",P91OP4SS16.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,16,Sunscreen
111,112,96,Compact Relief Kit,A personal first aid kit is recommended for ev...,P96OP4FA17.jpg,2011-02-15,16.4300,0.2800,EN,16.7100,4,Outdoor Protection,17,First Aid
112,113,97,Deluxe Family Relief Kit,A complete medical kit suitable for families w...,P96OP4FA17.jpg,2013-05-03,25.0000,0.2800,EN,25.2800,4,Outdoor Protection,17,First Aid
113,114,98,Calamine Relief,Use the Calamine Relief for allergic skin reac...,P98OP4FA17.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,17,First Aid


In [5]:
order_details_query = "SELECT * FROM Order_details"

order_details_result = export_cursor.execute(order_details_query)
order_details_fetch = order_details_result.fetchall()
order_details_columns = [column[0] for column in order_details_result.description]
order_details = pd.DataFrame.from_records(order_details_fetch, columns=order_details_columns)

# Dropping current_value and last_updated columns
order_details = order_details.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order_details

,ORDER_DETAILS_SK,ORDER_DETAILS_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_UNIT_ID_unit
0,1,100000,16,257.6000,360.6400,4405,112,1
1,2,100001,20,322.0000,450.8000,5008,112,2
2,3,100002,24,386.4000,540.9600,4394,112,3
3,4,100003,18,289.8000,405.7200,4396,112,4
4,5,100004,20,322.0000,450.8000,4382,112,5
...,...,...,...,...,...,...,...,...
43058,43059,99995,146,402.9600,810.3000,4402,111,43059
43059,43060,99996,172,474.7200,954.6000,4400,111,43060
43060,43061,99997,192,529.9200,1065.6000,5148,111,43061
43061,43062,99998,192,529.9200,1065.6000,4384,111,43062


In [6]:
order_query = "SELECT * FROM [Order]"

order_result = export_cursor.execute(order_query)
order_fetch = order_result.fetchall()
order_columns = [column[0] for column in order_result.description]
order = pd.DataFrame.from_records(order_fetch, columns=order_columns)

# Dropping current_value and last_updated columns
order = order.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order

,ORDER_SK,ORDER_order_number,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
0,1,1153,7,Sales visit
1,2,1154,4,E-mail
2,3,1155,7,Sales visit
3,4,1156,5,Web
4,5,1157,2,Telephone
...,...,...,...,...
5355,5356,9475,4,E-mail
5356,5357,9476,5,Web
5357,5358,9477,2,Telephone
5358,5359,9478,7,Sales visit


## Database connectie sluiten

In [7]:
export_cursor.close()
export_conn.close()

## Tabellen samenvoegen

In [8]:
combined_data = pd.merge(order_details, product, left_on='ORDER_DETAILS_PRODUCT_NUMBER_product', right_on='PRODUCT_SK')
combined_data = pd.merge(combined_data, order, left_on='ORDER_DETAILS_ORDER_NUMBER_order', right_on='ORDER_SK')

combined_data

,ORDER_DETAILS_SK,ORDER_DETAILS_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_UNIT_ID_unit,PRODUCT_SK,PRODUCT_number,...,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en,ORDER_SK,ORDER_order_number,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
0,1,100000,16,257.6000,360.6400,4405,112,1,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4405,8462,5,Web
1,2,100001,20,322.0000,450.8000,5008,112,2,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,5008,9111,2,Telephone
2,3,100002,24,386.4000,540.9600,4394,112,3,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4394,8451,2,Telephone
3,4,100003,18,289.8000,405.7200,4396,112,4,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4396,8453,7,Sales visit
4,5,100004,20,322.0000,450.8000,4382,112,5,112,96,...,EN,16.7100,4,Outdoor Protection,17,First Aid,4382,8439,7,Sales visit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,43059,99995,146,402.9600,810.3000,4402,111,43059,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4402,8459,4,E-mail
43059,43060,99996,172,474.7200,954.6000,4400,111,43060,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4400,8457,4,E-mail
43060,43061,99997,192,529.9200,1065.6000,5148,111,43061,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,5148,9267,5,Web
43061,43062,99998,192,529.9200,1065.6000,4384,111,43062,111,95,...,EN,3.5000,4,Outdoor Protection,16,Sunscreen,4384,8441,4,E-mail


## Tabel opschonen

In [9]:
# dropping columns
keep_columns = ['ORDER_DETAILS_QUANTITY_quantity','PRODUCT_name_product','PRODUCT_PRODUCT_LINE_code', 'PRODUCT_PRODUCT_TYPE_code', 'ORDER_ORDER_METHOD_CODE_method_code']
combined_data = combined_data.loc[:, keep_columns]

combined_data

,ORDER_DETAILS_QUANTITY_quantity,PRODUCT_name_product,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code,ORDER_ORDER_METHOD_CODE_method_code
0,16,Compact Relief Kit,4,17,5
1,20,Compact Relief Kit,4,17,2
2,24,Compact Relief Kit,4,17,2
3,18,Compact Relief Kit,4,17,7
4,20,Compact Relief Kit,4,17,7
...,...,...,...,...,...
43058,146,Sun Shield,4,16,4
43059,172,Sun Shield,4,16,4
43060,192,Sun Shield,4,16,5
43061,192,Sun Shield,4,16,4


## Data voorbereiden
data omzetten naar bruikbare data voor het neurale netwerk

In [10]:
# encoding PRODUCT_name_product. Cannot use dummy encoding since there are too many unique values and only one value per row
combined_data['PRODUCT_name_product'] = combined_data['PRODUCT_name_product'].astype('category').cat.codes.astype('int32')

# turning everything into int32
combined_data['PRODUCT_PRODUCT_LINE_code'] = combined_data['PRODUCT_PRODUCT_LINE_code'].astype('int32')
combined_data['ORDER_DETAILS_QUANTITY_quantity'] = combined_data['ORDER_DETAILS_QUANTITY_quantity'].astype('int32')
combined_data['PRODUCT_PRODUCT_TYPE_code'] = combined_data['PRODUCT_PRODUCT_TYPE_code'].astype('int32')
combined_data['ORDER_ORDER_METHOD_CODE_method_code'] = combined_data['ORDER_ORDER_METHOD_CODE_method_code'].astype('int32')

combined_data.dtypes

ORDER_DETAILS_QUANTITY_quantity        int32
PRODUCT_name_product                   int32
PRODUCT_PRODUCT_LINE_code              int32
PRODUCT_PRODUCT_TYPE_code              int32
ORDER_ORDER_METHOD_CODE_method_code    int32
dtype: object

## Data splitsen

In [11]:
# splitting the data in x and y
X = combined_data.drop(columns=['ORDER_DETAILS_QUANTITY_quantity'])
y = combined_data['ORDER_DETAILS_QUANTITY_quantity']

# splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# printing the sizes
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (34450, 4), X_test: (8613, 4), y_train: (34450,), y_test: (8613,)


In [12]:
# defining the amount of epochs and batch size
epochs = 500
batch_size = 50

## Tensorflow + keras
neural network using keras

### Model maken

In [13]:
# getting the amount of columns in order_details
amount_of_columns = len(combined_data.columns)-1 # needs -1 since one of the columns is the target column. Should be 4

# the model
model = Sequential()

# input layer, 4 nodes
model.add(Input(shape=(amount_of_columns,)))

# 1 hidden layer
model.add(Dense(32, activation='relu'))

# 2nd hidden layer
model.add(Dense(24, activation='relu'))

# output layer, 1 node
model.add(Dense(1, activation='linear')) # linear since we are predicting a number

# compile the model, so we can get metrics
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_logarithmic_error']) # adam uses gradient descent, doesnt seem to be able to use non-gradient descent based optimizer

### Model trainen

In [14]:
# training the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size) # batch size: after how many samples the weights are updated

Epoch 1/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 1s 495us/step - loss: 4248.2124 - mean_absolute_error: 38.1960 - mean_squared_logarithmic_error: 1.6311
Epoch 2/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 3850.6567 - mean_absolute_error: 35.6809 - mean_squared_logarithmic_error: 0.9404
Epoch 3/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 3397.8127 - mean_absolute_error: 33.5779 - mean_squared_logarithmic_error: 0.8420
Epoch 4/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - loss: 3079.1936 - mean_absolute_error: 32.6767 - mean_squared_logarithmic_error: 0.7941
Epoch 5/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 3593.7061 - mean_absolute_error: 32.9340 - mean_squared_logarithmic_error: 0.7706
Epoch 6/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 3124.9663 - mean_absolute_error: 32.5127 - mean_squared_logarithmic_error: 0.7968
Epoch 7/500
689/689 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - loss: 3183.8845 - mean_absolute_error: 32.5058 - mean_squared_logarithmic

### Model evalueren

In [15]:
# predicting the model
predictions = model.predict(X_test)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted'], index=X_test.index)

# Add the actual values to the DataFrame
predictions_df['Actual'] = y_test

predictions_df['Difference'] = predictions_df['Predicted'] - predictions_df['Actual']
predictions_df['Absolute Difference'] = abs(predictions_df['Difference'])

predictions_df

270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step


,Predicted,Actual,Difference,Absolute Difference
9130,49.914692,56,-6.085308,6.085308
27718,67.294991,36,31.294991,31.294991
41898,37.297047,26,11.297047,11.297047
15673,59.953228,24,35.953228,35.953228
2314,70.045815,46,24.045815,24.045815
...,...,...,...,...
38076,43.473000,46,-2.527000,2.527000
18731,57.706905,94,-36.293095,36.293095
2460,53.112999,56,-2.887001,2.887001
15719,65.113281,90,-24.886719,24.886719


In [16]:
# evaluating the model
loss, mae, mslr = model.evaluate(X_test, y_test)
average_difference = predictions_df['Absolute Difference'].mean()
print(f'Loss (Mean Squared Error): {loss}, Mean Absolute Error: {mae}, Mean Squared Logarithmic Error: {mslr}, Average Difference: {average_difference}')

270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - loss: 2152.5396 - mean_absolute_error: 28.1442 - mean_squared_logarithmic_error: 0.5398
Loss (Mean Squared Error): 2531.427734375, Mean Absolute Error: 28.308095932006836, Mean Squared Logarithmic Error: 0.5284209847450256, Average Difference: 28.308103987471117


## Pytorch
neural network using pytorch

### Model maken

In [17]:
# check if cuda is available
# to use gpu, torch needs to be installed with the correct cuda version
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the architecture of the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 16), # input layer with 16 output nodes
    nn.ReLU(),
    nn.Linear(16, 8), # 16 input nodes to 16 output nodes
    nn.ReLU(),
    nn.Linear(8, 1) # 16 input nodes to 1 output node
)

# Send the model to the device
model = model.to(device)

print(f'The model is using {device}')

The model is using cuda


### Model trainen

In [18]:
# Custom optimizer class which doesn't use gradient descent or backpropagation
class SimpleErrorOptimizerPyTorch(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01):
        defaults = dict(lr=lr)
        super(SimpleErrorOptimizerPyTorch, self).__init__(params, defaults)

    def step(self, closure=None):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                # Here we directly subtract the error from the parameters
                p.data -= group['lr'] * p.grad.data

In [19]:
# Converting the dataframes to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Define the loss function and the custom optimizer
criterion = nn.MSELoss()
optimizer = SimpleErrorOptimizerPyTorch(model.parameters(), lr=0.001)

# Define the DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Train the model
for epoch in range(epochs):  # 100 epochs
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/500, Loss: 857.3074951171875
Epoch 2/500, Loss: 2255.376953125
Epoch 3/500, Loss: 2648.048095703125
Epoch 4/500, Loss: 4011.901123046875
Epoch 5/500, Loss: 4907.1123046875
Epoch 6/500, Loss: 1939.5321044921875
Epoch 7/500, Loss: 3826.389404296875
Epoch 8/500, Loss: 1707.519287109375
Epoch 9/500, Loss: 3656.791748046875
Epoch 10/500, Loss: 1977.0999755859375
Epoch 11/500, Loss: 960.7280883789062
Epoch 12/500, Loss: 5553.470703125
Epoch 13/500, Loss: 1390.7059326171875
Epoch 14/500, Loss: 4125.02783203125
Epoch 15/500, Loss: 1343.591796875
Epoch 16/500, Loss: 821.4384765625
Epoch 17/500, Loss: 4062.966796875
Epoch 18/500, Loss: 3165.706787109375
Epoch 19/500, Loss: 18521.451171875
Epoch 20/500, Loss: 1898.5599365234375
Epoch 21/500, Loss: 3120.726806640625
Epoch 22/500, Loss: 11396.89453125
Epoch 23/500, Loss: 2055.083984375
Epoch 24/500, Loss: 1951.7884521484375
Epoch 25/500, Loss: 2329.10546875
Epoch 26/500, Loss: 1113.5135498046875
Epoch 27/500, Loss: 4089.701171875
Epoch 28/5

### Model evalueren

In [20]:
# Evaluate the model
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()
    loss = criterion(predictions, y_test_tensor)
print(f'Test Loss: {loss.item()}')

Test Loss: 3318.3095703125


In [21]:
# copy the predictions to the cpu
predictions_cpu_tensor = predictions.detach().cpu()
y_test_cpu_tensor = y_test_tensor.cpu()

# Convert the predictions to a numpy array
predictions_np = predictions_cpu_tensor.numpy()
y_test_np = y_test_cpu_tensor.numpy()

# Convert the numpy array to a pandas DataFrame
predictions_df = pd.DataFrame(predictions_np, columns=['Predicted'], index=X_test.index)

# Convert the actual values to a pandas DataFrame
actual_df = pd.DataFrame(y_test_np, columns=['Actual'], index=X_test.index)

# Concatenate the two dataframes along the column axis
result_df = pd.concat([predictions_df, actual_df], axis=1)

# adding difference and absolute difference
result_df['Difference'] = result_df['Predicted'] - result_df['Actual']
result_df['Absolute Difference'] = abs(result_df['Difference'])

result_df

,Predicted,Actual,Difference,Absolute Difference
9130,51.624249,56.0,-4.375751,4.375751
27718,51.624249,36.0,15.624249,15.624249
41898,51.624249,26.0,25.624249,25.624249
15673,51.624249,24.0,27.624249,27.624249
2314,51.624249,46.0,5.624249,5.624249
...,...,...,...,...
38076,51.624249,46.0,5.624249,5.624249
18731,51.624249,94.0,-42.375751,42.375751
2460,51.624249,56.0,-4.375751,4.375751
15719,51.624249,90.0,-38.375751,38.375751


In [ ]:
# Calculate the average difference
average_difference = result_df['Absolute Difference'].mean()
print(f'Average Difference: {average_difference}')

## Scratch
Een neural network maken zonder gebruik te maken van een library

### Model maken

In [22]:
class CustomNeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes

        # Initialize weights and biases
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5,
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5,
                                       (self.hidden_nodes, 1))
        self.bias_hidden = np.zeros(self.hidden_nodes)
        self.bias_output = np.zeros(1)

    @staticmethod
    def linear(x):
        return x

    def forward_pass(self, X):
        hidden_layer_input = np.dot(X, self.weights_input_to_hidden) + self.bias_hidden
        hidden_layer_output = self.linear(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.weights_hidden_to_output) + self.bias_output
        output_layer_output = self.linear(output_layer_input)

        return hidden_layer_output, output_layer_output

    @staticmethod
    def calculate_error(y, final_outputs):
        return y - final_outputs

    def update_weights(self, X, hidden_outputs, error, lr):
        epsilon = 1e-8
        # Simple rule for updating weights
        for i in range(self.weights_input_to_hidden.shape[0]):
            for j in range(self.weights_input_to_hidden.shape[1]):
                for k in range(X.shape[0]):
                    self.weights_input_to_hidden[i, j] += lr * error[k][0] * X[k, i] / (np.abs(hidden_outputs[k, j]) + epsilon)
                    
    def calculate_loss(self, y, final_outputs):
        # Calculate the Mean Squared Error loss
        return np.mean((y - final_outputs) ** 2)

        
    def train(self, X, y, lr, batch_size=50):
        # normalize the input data
        X = X / np.max(X, axis=0)
        
        # Initialize the loss
        loss = 0.0
        
        # Batch training
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            
            hidden_outputs, final_outputs = self.forward_pass(X_batch)
            error = self.calculate_error(y_batch, final_outputs)
            self.update_weights(X_batch, hidden_outputs, error, lr)
            loss = self.calculate_loss(y_batch, final_outputs)
        return loss

    def predict(self, X):
        _, final_outputs = self.forward_pass(X)
        return final_outputs

In [23]:
# Initialize the neural network
nn = CustomNeuralNetwork(input_nodes=4, hidden_nodes=8)

### Model trainen

In [24]:
# Convert pandas DataFrame to numpy array
X_train_np = X_train.values
y_train_np = y_train.values

# Train the model
for epoch in range(epochs):  # number of epochs
    loss = nn.train(X_train_np, y_train_np, lr=0.001, batch_size=batch_size)
    print(f'Epoch {epoch+1}/{epochs}, Loss(MSE): {loss}')

Epoch 1/500, Loss(MSE): 41.39129093631583
Epoch 2/500, Loss(MSE): 41.7811476793938
Epoch 3/500, Loss(MSE): 41.82521275104665
Epoch 4/500, Loss(MSE): 41.82275250923871
Epoch 5/500, Loss(MSE): 41.81702020019149
Epoch 6/500, Loss(MSE): 41.81250978254357
Epoch 7/500, Loss(MSE): 41.80912320812436
Epoch 8/500, Loss(MSE): 41.80652497484913
Epoch 9/500, Loss(MSE): 41.80444805017139
Epoch 10/500, Loss(MSE): 41.80281798882764
Epoch 11/500, Loss(MSE): 41.801581614892264
Epoch 12/500, Loss(MSE): 41.80054771944224
Epoch 13/500, Loss(MSE): 41.7996823722765
Epoch 14/500, Loss(MSE): 41.79896591069845
Epoch 15/500, Loss(MSE): 41.79838766021559
Epoch 16/500, Loss(MSE): 41.79791272785633
Epoch 17/500, Loss(MSE): 41.79752717213745
Epoch 18/500, Loss(MSE): 41.79721890302286
Epoch 19/500, Loss(MSE): 41.79697736425153
Epoch 20/500, Loss(MSE): 41.79679337149186
Epoch 21/500, Loss(MSE): 41.79666053311607
Epoch 22/500, Loss(MSE): 41.79656943126193
Epoch 23/500, Loss(MSE): 41.796522123429234
Epoch 24/500, Loss(M

### Model evalueren

In [25]:
# Generate predictions for the test data
predictions = nn.predict(X_test.values)

# Calculate the error
error = y_test.values - predictions

# Square the error
squared_error = np.square(error)

# Calculate the mean of the squared errors
mse = np.mean(squared_error)

print(f'Mean Squared Error: {mse}')

# Convert the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted'], index=X_test.index)
# adding the actual values to the dataframe
predictions_df['Actual'] = y_test
predictions_df['Difference'] = predictions_df['Predicted'] - predictions_df['Actual']
predictions_df['Absolute Difference'] = abs(predictions_df['Difference'])

predictions_df

Mean Squared Error: 8182748.722262755


,Predicted,Actual,Difference,Absolute Difference
9130,3572.949940,56,3516.949940,3516.949940
27718,1644.388599,36,1608.388599,1608.388599
41898,2772.833450,26,2746.833450,2746.833450
15673,1956.277276,24,1932.277276,1932.277276
2314,2133.148955,46,2087.148955,2087.148955
...,...,...,...,...
38076,3740.011614,46,3694.011614,3694.011614
18731,4136.185523,94,4042.185523,4042.185523
2460,3502.598164,56,3446.598164,3446.598164
15719,2158.480623,90,2068.480623,2068.480623


In [26]:
# Calculate the average difference
average_difference = predictions_df['Absolute Difference'].mean()
print(f'Average Difference: {average_difference}')

Average Difference: 2621.7266332248496
